# Rota mais valiosa
Instância Um grafo completo G = (V, A) com distâncias da, a ∈ A, entre os vértices, prêmios pv,
v ∈ V , nos vértices e um vértice de origem vo ∈ V .
Solução Uma rota R = (vo, v1, v2, . . . , vk, vo) inciando e terminando em vo com vértices intermediários vi ∈ V . Cada vértice em V \ {vo} pode ser contido no máximo um vez em R.
Objetivo Minimizar o valor total v(R) = dvo,v1 +
∑
i∈[k−1] dvi,vi+1 + dvk,vo −
∑
i∈[k]
pvi da rota.

## Variáveis
- D -> Matriz de adjacência com o peso de cada aresta. Tamanho K,K
- A -> Matriz de adjacência com a booleano informando se a aresta pertence ou não a solução. Tamanho K,K
- P -> Lista com o peso (prêmio) de cada vértice. Tamanho K
- V -> Lista com a booleano informando se o vértice foi visitado ou não. Tamanho K
- U -> Lista auxiliar para eliminação de subrotas. Tamanho K

## Modelagem
Minimizar 
    $\sum_{i=1}^{N}\sum_{j=1}^{K} D_{(i,j)} * A_{(i,j)} - \sum_{i=1}^{K} P_{(i)} * V_{(i)}$


Sujeito:
    
 1. A aresta i,j pode (1) ou não (0) estar contido na solução: 
     
     $A_{(i,j)} \in \{0,1\},\ \forall_{i,j} \in \{1,..,K\}$ 
 
 
 2. O vértica i pode (1) ou não (0) estar contido em algumas das arestas da solução:
     
     $V_{(i)} \in \{0,1\}, \forall_{i,j} \in \{1,..,K\}$ 
 
 
 3. A aresta que sai do vértice 1, que é vértice de partida, deve obrigatoriamente estar contido na solução
     
     $\sum_{j=1}^{K} A_{1,j} = 1$
 
 
 4. A aresta que chega no vértice 1, que é vértice de destino, deve obrigatoriamente estar contido na solução
     
     $\sum_{i=1}^{K} A_{i,1} = 1$
 
 
 5. Vinculação dos vértice visitados com os prêmios destes.
     
     $\sum_{j=1}^{K} A_{i,j} = V_{i},\ \forall_{i} \in \{1,..,K\}$
 
 
 6. Um vértice não pode estar contido como destino em mais de 1 aresta da solução
     
     $\sum_{i=1}^{K} A_{i,j} <= 1,\ \forall_{j} \in \{1,..,K\}$
 
 
 7. Um vértice não pode estar contido como partida em mais de 1 aresta da solução
     
     $\sum_{j=1}^{K} A_{i,j} <= 1,\ \forall_{i} \in \{1,..,K\}$
     
     
 8. Um aresta não pode conter o mesmo vértice como partida e destino
     
     $A[i,i] = 0,\ \forall_{i} \in \{1,..,K\}$
 
 
 9. Restrições para eliminação de subrotas
     
     $U_{i} - U_{j} + K*A_{i,j} \le K - 1,\ \forall_{i,j} \in \{1,..,K\}, i \neq j $
     
     $2 \le U_{i},\ \forall_{i} \in \{2,..,K\}$
     
     $U_{i} \le K,\ \forall_{i} \in \{2,..,K\}$
     
     $ U_{1} == 1 $

## Código

In [1]:
using NBInclude
using GLPK
using JuMP
@nbinclude("parser.ipynb")

In [2]:
# file = "instancias/A-n32-k5.vrp"
# v, P = parse_file(file)
# D = get_distancias(v)
# k = length(P);
# #k=6;
# m = Model(GLPK.Optimizer)
# @variable(m, A[1:k, 1:k], Bin)
# @variable(m, V[1:k], Bin);
# @variable(m, u[1:k]);

# @objective(m, Min, sum(D[1:k] .* A[1:k]) - sum(P[1:k] .* V[1:k]));

# @constraints(m, begin
#     #3
#     sum(A[i,1] for i=1:k) == 1
    
#     #4
#     sum(A[1,j] for j=1:k) == 1
    
#     #5
#     [i=1:k], sum(A[i,j] for j=1:k) == V[i] 
        
#     #6
#     [i=1:k], sum(A[i,j] for j=1:k) <= 1
        
#     #7
#     [j=1:k], sum(A[i,j] for i=1:k) <= 1
    
#     #8
#     [i=1:k], A[i,i] == 0
    
#     #9    
#     [i=2:k,j=2:k; i!=j], u[i] - u[j] + k*A[i,j] <= k - 1
#     [i=2:k], 2 <= u[i]
#     [i=2:k], u[i] <= k
#     u[1] == 1
#     end
# )

In [3]:
# for i in 1:k
#     if 1.0 in JuMP.value.(A)[1,:]
#         println(i," ", argmax(JuMP.value.(A)[i,:]))
#     end
# end

In [4]:
function otimiza(arquivo)
    v, P = parse_file(arquivo)
    D = get_distancias(v)
    k = length(P);
    #k=6;
    m = Model(GLPK.Optimizer)
    @variable(m, A[1:k, 1:k], Bin)
    @variable(m, V[1:k], Bin);
    @variable(m, u[1:k]);

    @objective(m, Min, sum(D[1:k] .* A[1:k]) - sum(P[1:k] .* V[1:k]));

    @constraints(m, begin
        #3
        sum(A[i,1] for i=1:k) == 1

        #4
        sum(A[1,j] for j=1:k) == 1

        #5
        [i=1:k], sum(A[i,j] for j=1:k) == V[i] 

        #6
        [i=1:k], sum(A[i,j] for j=1:k) <= 1

        #7
        [j=1:k], sum(A[i,j] for i=1:k) <= 1

        #8
        [i=1:k], A[i,i] == 0

        #9    
        [i=2:k,j=2:k; i!=j], u[i] - u[j] + k*A[i,j] <= k - 1
        [i=2:k], 2 <= u[i]
        [i=2:k], u[i] <= k
        u[1] == 1
        end
    )
    
    optimize!(m)
    @show objective_value(m)
    return m, A
end

otimiza (generic function with 1 method)

In [5]:
# teste
for file in readdir("instancias")
    println(file)
end

A-n32-k5.vrp
A-n80-k10.vrp
B-n31-k5.vrp
B-n78-k10.vrp
E-n101-k14.vrp
E-n76-k8.vrp
F-n72-k4.vrp
G-n262-k25.vrp
M-n101-k10.vrp
M-n200-k17.vrp
P-n101-k4.vrp
P-n76-k5.vrp


In [15]:
file = "A-n32-k5.vrp"

m, A = otimiza(file);

open("resultados/" *file * ".log", "w") do file
    for i in 1:size(A)[1]
        if 1.0 in JuMP.value.(A)[1,:]
            println(file, i," ", argmax(JuMP.value.(A)[i,:]))
        end
    end
end

objective_value(m) = -394.0


In [ ]:
file = "A-n80-k10.vrp"

m, A = otimiza(file);

open("resultados/" *file * ".log", "w") do file
    for i in 1:size(A)[1]
        if 1.0 in JuMP.value.(A)[1,:]
            println(file, i," ", argmax(JuMP.value.(A)[i,:]))
        end
    end
end